In [37]:
import pandas as pd

In [38]:
test_unseen = pd.read_csv('../data/test_unseen.csv')
train = pd.read_csv('../data/train.csv')
val_seen = pd.read_csv('../data/val_seen.csv')
val_unseen = pd.read_csv('../data/val_unseen.csv')
courses = pd.read_csv('../data/courses.csv').fillna('')
users = pd.read_csv('../data/users.csv').fillna('')

In [39]:
popular_subgroup = {}
for sub in train_group.subgroup:
    for s in sub.split(' '):
        if(s == ''):
            continue
        try:
            popular_subgroup[s] += 1
        except:
            popular_subgroup[s] = 1
for sub in val_seen_group.subgroup:
    for s in sub.split(' '):
        if(s == ''):
            continue
        try:
            popular_subgroup[s] += 1
        except:
            popular_subgroup[s] = 1
for sub in val_unseen_group.subgroup:
    for s in sub.split(' '):
        if(s == ''):
            continue
        try:
            popular_subgroup[s] += 1
        except:
            popular_subgroup[s] = 1
popular_subgroup_list = sorted(list(popular_subgroup.items()), key=lambda x:x[1], reverse=True)
popular_subgroup_index = [x[0] for x in popular_subgroup_list]

In [40]:
popular_course = {}
for course in train.course_id:
    for c in course.split(' '):
        try:
            popular_course[c] += 1
        except:
            popular_course[c] = 1
for course in val_seen.course_id:
    for c in course.split(' '):
        try:
            popular_course[c] += 1
        except:
            popular_course[c] = 1
for course in val_unseen.course_id:
    for c in course.split(' '):
        try:
            popular_course[c] += 1
        except:
            popular_course[c] = 1
for x in list(set(courses['course_id'].to_list()).difference(set(popular_course.keys()))):
    popular_course[x] = 0
popular_course_list = sorted(list(popular_course.items()), key=lambda x:x[1], reverse=True)

In [41]:
courses_interests = {}
for c, i in zip(courses.course_id, courses['groups+subgroups']):
    courses_interests[c] = i.split(',')

In [42]:
users_interests = {}
for u, i in zip(users.user_id, users.interests):
    users_interests[u] = i.split(',')

In [43]:
popular_course_with_interest = {}
for pc in popular_course_list:
    popular_course_with_interest[pc[0]] = courses_interests[pc[0]]
popular_course_with_interest = list(popular_course_with_interest.items())

In [44]:
with open('./submit_unseen_course.csv', 'w') as f:
    f.write('user_id,course_id\n')
    for user in test_unseen.user_id:
        self_interest = set(users_interests[user])
        if(len(self_interest) == 0):
            rec = popular_course_with_interest[:50]
        else:
            temp = [[] for x in range(len(self_interest) + 1)]
            for c in popular_course_with_interest:
                c_interest = set(c[1])
                temp[len(self_interest.intersection(c_interest))].append(c[0])
            rec = [item for sublist in temp[::-1] for item in sublist][:50]

        f.write('{},{}\n'.format(user, ' '.join(rec)))